In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
dt = 0.2                        # Cada cuantos segundos samplea el sensor
N_samples = 100                 # Cuantos sampleos queremos observar el sistema

tiempo = np.array([dt*i for i in range(N_samples)]) # No hace falta para Kalman, es para graficar (A Kalman solo le importa el cambio de tiempo entre cada medición/estimación. Eso ya es "dt")
Zsaved = np.zeros((N_samples,1))                    # Aca se van a ir guardando las mediciones crudas, para poder graficarlas nomas.
Xsaved = np.zeros((N_samples,2))                    # Aca se van a ir guardando las estimaciones de Kalman (despues de cada medición) de posición y de velocidad

In [ ]:
# Estado real del auto. Esto no lo sabemos
x_pos_inicial=0         # Solo estamos midiendo 1 dimension. Distancia en un eje.
x_vel=1                 # Solo estamos midiendo 1 dimension. Velocidad en ese eje.
ruido_proceso = 0.2     # Ruido de la posición del auto. Tiene velocidad constante en el largo plazo, pero por terreno imperfecto aveces se adelanta o atrasa un poco
ruido_gps = 1
x = np.array([[x_vel * t + np.random.normal(0, ruido_proceso), x_vel] for t in range(N_samples)]) 

In [ ]:
def get_medicion1(k):                       # El sensor que usas para medir posición del auto
    return x[k] + np.random.normal(0, ruido_gps)    # Toma la posicion real del auto en momento k y agrega ruido porque no mide perfectamente

In [ ]:
def kalman_filter(estado_inicial, matriz_covarianza_inicial, ruido_proceso_pos, ruido_proceso_vel, varianza_mediciones):
    
    
    while True:
        # Initial prediction step
        x_hat_predicción_lineal = np.dot(F, x_hat)
        P_minus = np.dot(np.dot(F, P), F.T) + Q

        # Estimation step
        K = np.dot(np.dot(P_minus, H.T), np.linalg.inv(np.dot(np.dot(H, P_minus), H.T) + R))
        z = yield x_hat_predicción_lineal[0]  # (yield es como return pero la prox que llamas la funcion arranca desde aca, entonces se acuerda de todas las variables)

        x_hat = x_hat_predicción_lineal + np.dot(K, (z - np.dot(H, x_hat_predicción_lineal)))
        P = np.dot((np.eye(2) - np.dot(K, H)), P_minus)


def kalman_filter_one_step(x_hat_previo, P, F, H, Q, R, medición):
    
    # Step 1: predicción preliminar de posición y velocidad según modelo lineal.
    x_hat_minus = np.dot(F, x_hat_previo)
    P_minus = np.dot(np.dot(F, P), F.T) + Q

    # Step 2: Estimar posición y velocidad
    K = np.dot(np.dot(P_minus, H.T), np.linalg.inv(np.dot(np.dot(H, P_minus), H.T) + R))
    x_hat = x_hat_minus + np.dot(K, (medición - np.dot(H, x_hat_minus)))
    P = np.dot((np.eye(2) - np.dot(K, H)), P_minus)

    return x_hat, P

In [ ]:
x_hat_inicial = np.array([x_pos_inicial,x_vel]) # Una primera 'estimacion' de [posición,velocidad]. Si sabes la posición y velocidad inicial del auto metes eso, sino tenes que tirarle a pegar, y Kalman va funcionar igual porque se va ir ajustando a las mediciones.

P = np.eye(2)                                   # Matriz de covarianza inicial (le tiras a pegar, corresponde a la incerteza que tenes sobre tus estimaciones creo)

F = np.array([[1, dt],                          # Matriz de transición de estados (supone velocidad constante). Significa que predecis linealmente. posición_nueva = 1*posicion_anterior + (cambio_de_tiempo * velocidad_anterior) , velocidad_nueva = 0*posición_anterior + 1*velocidad_anterior
              [0, 1 ]])

H = np.array([[1, 0]])                          # "Matriz de observación" ([1, 0] significa que solo tenes mediciones de posición, no de velocidad)

ruido_proceso_pos = 0.2
ruido_proceso_vel = 0
Q = np.array([[ruido_proceso_pos, 0],           # "Process noise covariance matrix": corresponde a lo que pensas que es el ruido real de nuestro sistema en la posición y velocidad del auto (Si lo sabes genial, sino predecis)
              [0, ruido_proceso_vel]])

R = np.array([1])                               # "Measurement noise covariance matrix": Ruido de las mediciones, en nuestro caso solo medimos posición, es un escalar no matriz.


# Finalmente las estimaciones paso a paso del KF, segun las mediciones en cada paso.
for i in range(N_samples):
    medición = get_medicion1(i)
    Zsaved[i] = medición

    if i==0:
        Xsaved[i] = kalman_filter_one_step(x_hat_inicial, P, F, H, Q, R, medición)
    else:
        Xsaved[i] = kalman_filter_one_step(Xsaved[i-1], P, F, H, Q, R, medición)